<a href="https://colab.research.google.com/github/Gustavo-Ros/Simulacion-1/blob/main/Inventario.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import random as rnd
import numpy as np
from tabulate import tabulate
#import matplotlib.pyplot as plt
#import math as mt

In [10]:
def demanda():
  prob=[0,0.01,0.025,0.045,0.065, 0.087,0.11,0.135,0.162,0.19,0.219,0.254,0.299,0.359,0.424,0.494,0.574,0.649,0.719,0.784,0.844, 0.894,0.934,0.964,0.980,0.995,1]
  u=rnd.random()
  for i in range(len(prob)):
      if prob[i-1] <= u <= prob[i]:
          #print('[',prob[i-1],',',prob[i],']')
          x=i+34
          #print(x)
          break
  return x,u

In [ ]:
#x=demanda()

In [3]:
def demanda_ajustada(x,mes):
  factor_mes=[1.2, 1, 0.9, 0.8, 0.8 ,0.7, 0.8, 0.9, 1, 1.2, 1.3, 1.4]
  j=1
  for j in range(len(factor_mes)):
    if mes==j:
      x=round(x*factor_mes[j-1])
  return x

In [42]:
def entrega(entrega,cont,carga):
  v=rnd.random()
  if cont==entrega:
    cont=-1
    carga=False

  if carga==False:
    carga=True
    if 0<= v < 0.3:
      entrega=1
    elif 0.3 <= v < 0.7:
      entrega=2
    elif 0.7 <= v < 1:
      entrega=3
  else:
    cont+=1


  return entrega,cont,carga


In [35]:
conta=0
final=89
carga=False
if final<=100:
  if inv_final<=100:
    ent,cont,carga=entrega(cont,carga)
print(t,conta,carga)

3 0 True


In [49]:
inv_inicial=150
ent=0
cont=-1
data=[]
carga=False
for m in range(12):
  mes=m+1
  dem,val=demanda()                         #DEMANDA
  dem_ajus=demanda_ajustada(dem,mes)        #DEMANDA AJUSTADA
  inv_final=inv_inicial-dem_ajus            #DEMANDA FINAL
  inv_men_prom=(inv_inicial+dem_ajus)/2

  if inv_final<=100:
    ent,cont,carga=entrega(ent,cont,carga)

  if cont==ent:
    inv_inicial+=200
  if inv_final<0:
    inv_final=0

  if carga==False:
    data.append([f"{mes}",f"{inv_inicial}", f"{val}" ,f"{dem}", f"{dem_ajus}", f"{inv_final}", f"{ent}", f"{inv_men_prom}"])
  elif carga==True:
    data.append([f"{mes}",f"{inv_inicial}", f"{val}" ,f"{dem}", f"{dem_ajus}", f"{inv_final}", "", f"{inv_men_prom}"])
  inv_inicial=inv_final
headers = ['Mes', 'Inventario inicial' ,'No. aleatorio','Demanda' , 'Demanda ajustada' , 'Inventario final', 'Entrega', 'Inventario mensual promedio']
print(tabulate(data, headers=headers, tablefmt="fancy_grid",showindex=False))


╒═══════╤══════════════════════╤═════════════════╤═══════════╤════════════════════╤════════════════════╤═══════════╤═══════════════════════════════╕
│   Mes │   Inventario inicial │   No. aleatorio │   Demanda │   Demanda ajustada │   Inventario final │ Entrega   │   Inventario mensual promedio │
╞═══════╪══════════════════════╪═════════════════╪═══════════╪════════════════════╪════════════════════╪═══════════╪═══════════════════════════════╡
│     1 │                  150 │       0.225779  │        45 │                 54 │                 96 │           │                         102   │
├───────┼──────────────────────┼─────────────────┼───────────┼────────────────────┼────────────────────┼───────────┼───────────────────────────────┤
│     2 │                   96 │       0.101506  │        40 │                 40 │                 56 │           │                          68   │
├───────┼──────────────────────┼─────────────────┼───────────┼────────────────────┼────────────────────┼──